Constant-Q 특징을 이용한 딥러닝 오디오 분류
   - 오디오 데이터를 spectrogram으로 가공하면 파장과 세기를 가진 이미지(2차원 배열)가 생성
   - 이 spectrogram을 CNN 이미지 분류를 통해 각 악기 소리를 분류

In [1]:
import numpy as np
import itertools
import librosa
import librosa.display 
import IPython.display as ipd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
plt.style.use('seaborn-white')

ModuleNotFoundError: No module named 'librosa'

In [1]:
midi_file = './GeneralMidi.wav'
instruments = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
num_notes = 50
sec = 2

audio = []
inst = []
for inst_idx, note in itertools.product(range(len(instruments)), range(num_notes)):
    instrument = instruments[inst_idx]
    offset = (instrument*num_notes*sec + (note*sec))
    print('instrument : {}, note: {}, offset : {}'.format(instrument, note, offset))
    y, sr = librosa.load(midi_file, sr=None, offset=offset, duration=2.0)
    audio.append(y)
    inst.append(inst_idx)

NameError: name 'itertools' is not defined

In [ ]:
from sklearn.preprocessing import OneHotEncoder

audio_cqt = []
for y in audio:
    ret = librosa.cqt(y, sr, hop_length=1024, n_bins=24*7, bins_per_octave=24)
    ret = np.abs(ret)

cqt_np = np.array(audio_cqt, np.float32)
cqt_np = cqt_np.reshape((500, 168 * 87))
cqt_array = np.expand.dims(cqt_np, -1)

# one-hot encoding
inst_np = np.array(inst, np.int16)
inst_cat = to_categorical(inst_np)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(cqt_np)

from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(cqt_array, inst_cat, test_size=0.2)

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

DNN 모델 구성

In [ ]:
def model_build():
    model = Sequential()

    input = Input(shape=(14616, ), name='input')
    output = Dense(512, activation='relu', name='hidden1')(input)
    output = Dense(256, activation='relu', name='hidden2')(output)
    output = Dense(128, activation='relu', name='hidden3')(output)
    output = Dense(10, activation='softmax', name='output')(output)

    model = Model(inputs=[input], outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    return model

In [ ]:
model = model_build

model = model_build